In [5]:
import warnings
import os
import sys

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(root_path)

from core.data_sources.clob import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

# Constants
CONNECTOR_NAME = "binance"
INTERVAL       = "1s"
TRADING_PAIR   = "BTC-USDT"
DAYS           = 3

2025-05-06 08:38:54,539 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f1a4066e3f0>


In [ ]:
await clob.get_candles_last_days(CONNECTOR_NAME, TRADING_PAIR, INTERVAL, DAYS)

In [ ]:
path = '/home/dominhnhat/quants-lab/test_data'
clob.dump_candles_cache(path)
btc_candles = clob.candles_cache["binance", "BTC-USDT", "1s"]

In [8]:
btc_candles.data

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume
timestamp,,,,,,,,,,
2025-05-03 08:38:54,1746261534,96381.72,96381.72,96381.72,96381.72,0.00021,20.2401612,1,0.00021,20.2401612
2025-05-03 08:38:55,1746261535,96381.71,96381.71,96381.71,96381.71,0.00241,232.2799211,3,0,0
2025-05-03 08:38:56,1746261536,96381.72,96381.72,96381.72,96381.72,0.00009,8.6743548,1,0.00009,8.6743548
2025-05-03 08:38:57,1746261537,96381.72,96381.72,96381.71,96381.71,0.00396,381.6715902,2,0.00186,179.2699992
2025-05-03 08:38:58,1746261538,96381.72,96381.72,96381.72,96381.72,0.00945,910.807254,3,0.00945,910.807254
...,...,...,...,...,...,...,...,...,...,...
2025-05-06 08:38:50,1746520730,94188.26,94188.27,94188.26,94188.27,0.01604,1510.7798337,13,0.01433,1349.7179091
2025-05-06 08:38:51,1746520731,94188.26,94188.26,94188.26,94188.26,0.00452,425.7309352,1,0,0
2025-05-06 08:38:52,1746520732,94188.27,94188.27,94188.27,94188.27,0.01749,1647.3528423,4,0.01749,1647.3528423


In [9]:
df = btc_candles.data
df['side'] = 1
from core.backtesting.triple_barrier_method import triple_barrier_method

df_with_tbm = triple_barrier_method(df[:1000], tp=5.0, sl=5.0, tl=300, std_span=100)

In [12]:
df_with_tbm.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume',
       'quote_asset_volume', 'n_trades', 'taker_buy_base_volume',
       'taker_buy_quote_volume', 'side', 'target', 'tl', 'stop_loss_time',
       'take_profit_time', 'close_time', 'close_type', 'ret', 'real_class',
       'tp', 'sl'],
      dtype='object')

In [16]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Create a copy of the dataframe to avoid modifying the original
df_processed = df_with_tbm.copy()
df_processed.reset_index(inplace=True, drop=True)

# 1. Remove unnecessary columns
columns_to_drop = [
    'timestamp',
    'taker_buy_base_volume',
    'volume',
    'close_time',
    'real_class',
    'ret',
    'tp',
    'sl',
    'take_profit_time',
    'stop_loss_time',
    'side'
]
df_processed = df_processed.drop(columns=columns_to_drop)


In [17]:
df_processed

,open,high,low,close,quote_asset_volume,n_trades,taker_buy_quote_volume,target,tl,close_type
0,96390.28,96390.28,96390.27,96390.28,680.5153495,10,417.3699124,0.00010659,2025-05-03 08:45:33,0
1,96390.28,96390.28,96390.27,96390.27,336.4020446,5,22.1697644,0.00010524,2025-05-03 08:45:34,0
2,96390.27,96390.28,96390.27,96390.28,52.0507471,2,12.5307364,0.00010386,2025-05-03 08:45:35,0
3,96390.27,96390.28,96390.27,96390.28,262.1815571,2,218.8059356,0.00010246,2025-05-03 08:45:36,0
4,96390.27,96390.27,96390.27,96390.27,129.1629618,1,0,0.00010102,2025-05-03 08:45:37,0
...,...,...,...,...,...,...,...,...,...,...
896,96283.31,96283.32,96283.31,96283.31,146.3506378,3,63.5469912,0.00002115,2025-05-03 09:00:29,0
897,96283.31,96283.31,96283.31,96283.31,0,0,0,0.00001841,2025-05-03 09:00:30,0
898,96283.32,96283.32,96283.32,96283.32,1896.781404,1,1896.781404,0.00001512,2025-05-03 09:00:31,0
899,96283.32,96283.32,96283.32,96283.32,55.8443256,2,55.8443256,0.00001074,2025-05-03 09:00:32,0


In [18]:
# 2. Convert prices to returns
price_columns = ['open', 'high', 'low', 'close']
for col in price_columns:
    df_processed[f'{col}_ret'] = df_processed[col].pct_change()
df_processed = df_processed.drop(columns=price_columns)

# 3. Create buy/sell volume ratio
df_processed['buy_volume_ratio'] = (
    df_processed['taker_buy_quote_volume']
    / df_processed['quote_asset_volume']
)
df_processed = df_processed.drop(columns=['taker_buy_quote_volume'])

# 4. Drop any rows with NaN values (first row will have NaN due to returns calculation)
df_processed = df_processed.dropna()

# 5. Prepare columns for scaling
columns_to_scale = [
    'open_ret', 'high_ret', 'low_ret', 'close_ret',
    'quote_asset_volume', 'n_trades', 'buy_volume_ratio', 'target'
]

# 6. Apply StandardScaler
scaler = StandardScaler()
df_processed[columns_to_scale] = scaler.fit_transform(
    df_processed[columns_to_scale]
)

# Show the first few rows of the processed dataset
print('Processed dataset shape:', df_processed.shape)
df_processed.head()


Processed dataset shape: (830, 10)


,quote_asset_volume,n_trades,target,tl,close_type,open_ret,high_ret,low_ret,close_ret,buy_volume_ratio
1,-0.2285509,-0.19660255,0.13706941,2025-05-03 08:45:34,0,0.06498159,0.06310132,0.07919563,0.07295194,-1.30289243
2,-0.23725378,-0.27204412,0.10911157,2025-05-03 08:45:35,0,0.05862525,0.06310132,0.07919563,0.08521925,-0.88586711
3,-0.2308225,-0.27204412,0.08066225,2025-05-03 08:45:36,0,0.06498159,0.06310132,0.07919563,0.0790856,0.53051218
4,-0.23489368,-0.29719131,0.05163928,2025-05-03 08:45:37,0,0.06498159,0.05691866,0.07919563,0.07295194,-1.46008376
5,-0.23722428,-0.29719131,0.02208078,2025-05-03 08:45:38,0,0.06498159,0.06310132,0.07919563,0.0790856,-1.46008376
